In [1]:
# !pip install transformers
# !pip install -q torch torchvision torchaudio
# !pip install supabase
# !pip install dotenv

In [2]:
from google.colab import files,drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import sys
sys.path.append('/gdrive/MyDrive/finn-project/be-ai-model')

from data_processing import get_csv_data, merge_data
from sentiment_model import analyze_sentiment_with_progress_for_training, add_integer_column, sums_sentiment_score_for_7_days, update_sentiment_score_in_db
from lstm_model import get_scale_data, get_scale_data_with_fit, create_sequences_for_train, compile_model, train_model, predict_prices, create_sequences_for_prod
from final_result import compare_prices_with_graph
from config import STOCK_LIST
import pandas as pd
import numpy as np
import pickle

from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [ ]:
def get_existing_model(path):
  load_path = '/gdrive/MyDrive/finn-project/models/finn_model.h5'
  return load_model(load_path)

In [ ]:
def train_model(stock_code):
    news_train_df = get_csv_data(f"/gdrive/MyDrive/finn-project/news/{stock_code}_news_train.csv")
    news_test_df = get_csv_data(f"/gdrive/MyDrive/finn-project/news/{stock_code}_news_test.csv")

    news_train_df = analyze_sentiment_with_progress_for_training(news_train_df)
    news_test_df = analyze_sentiment_with_progress_for_training(news_test_df)

    stock_train_df = get_csv_data(f"/gdrive/MyDrive/finn-project/price/{stock_code}_prices_train.csv")
    stock_test_df = get_csv_data(f"/gdrive/MyDrive/finn-project/price/{stock_code}_prices_test.csv")

    stock_train_df['date'] = pd.to_datetime(stock_train_df['date']).dt.strftime('%Y-%m-%d')
    stock_test_df['date'] = pd.to_datetime(stock_test_df['date']).dt.strftime('%Y-%m-%d')

    merged_train = merge_data(news_train_df, stock_train_df)
    merged_test = merge_data(news_test_df, stock_test_df)

    merged_train['sentiment_influence'] = 0.0
    merged_test['sentiment_influence'] = 0.0

    merged_train = add_integer_column(merged_train)
    merged_test = add_integer_column(merged_test)

    sums_sentiment_score_for_7_days(merged_train)
    sums_sentiment_score_for_7_days(merged_test)

    scaler = MinMaxScaler()

    features = ['sentiment_influence', 'open', 'high', 'low', 'adjClose', 'volume']
    target   = 'close'

    merged_train = merged_train[ features + [target] ].dropna()
    merged_test  = merged_test[ features + [target] ].dropna()

    train_scaled = get_scale_data_with_fit(scaler, merged_train)
    test_scaled = get_scale_data(scaler, merged_test)

    X_train, y_train = create_sequences_for_train(train_scaled)
    X_test, y_test = create_sequences_for_train(test_scaled)


    model = compile_model(X_train)
    model = train_model(model, X_train, y_train, X_test, y_test)

    y_pred_scaled = predict_prices(model, X_test)

    features = ['sentiment_influence', 'open', 'high', 'low', 'adjClose', 'volume']
    target   = 'close'
    all_cols = features + [target]
    target_col_index = all_cols.index(target)

    num_features = len(features)
    dummy_array = np.zeros((len(y_pred_scaled), len(all_cols)))
    # 'close' 위치(6번 인덱스)에 예측된 값을 삽입
    dummy_array[:, target_col_index] = y_pred_scaled.ravel()
    # Scaler를 이용해 전체 배열을 역변환하고, 'close' 컬럼만 추출
    y_pred_actual = scaler.inverse_transform(dummy_array)[:, target_col_index]

    # y_test도 역변환
    dummy_array_test = np.zeros((len(y_test), len(all_cols)))
    dummy_array_test[:, target_col_index] = y_test.ravel()
    y_test_actual = scaler.inverse_transform(dummy_array_test)[:, target_col_index]

    next_day_predicted_close = y_pred_actual[-1]
    print(f"{stock_code}의 오늘의 예측 종가: ${next_day_predicted_close:.4f}")
    # compare_prices_with_graph(y_test_actual, y_pred_actual)

    # 모델 저장
    save_path = f'/gdrive/MyDrive/finn-project/be-ai-model/models/{stock_code}_finn_model.keras'
    model.save(save_path)

    # scaler 저장
    scaler_path = f'/gdrive/MyDrive/finn-project/be-ai-model/models/{stock_code}_finn_scaler.pkl'
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)

In [ ]:
def main():
    for stock_code in STOCK_LIST:
        train_model(stock_code)

In [ ]:
if __name__ == '__main__':
    main()